In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 분류
from sklearn.neighbors import KNeighborsClassifier

# 모델 평가 지표 모아놓음, 
# 2진분류 confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# load_dataset 함수를 사용하여 데이터프레임으로 변환
# seaborn 으로 titanic 데이터 불러옴 
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
df.shape

(891, 15)

In [5]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [8]:
rdf = df.drop(["deck", "embark_town"], axis = 1)

In [9]:
rdf.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype='object')

In [10]:
rdf.shape

(891, 13)

In [11]:
rdf = rdf.dropna(subset = ["age"], axis = 0)

In [12]:
rdf.shape

(714, 13)

In [13]:
rdf["embarked"].value_counts()

embarked
S    554
C    130
Q     28
Name: count, dtype: int64

In [14]:
most_freq = rdf["embarked"].value_counts().idxmax()
most_freq

'S'

In [15]:
rdf["embarked"].fillna(most_freq, inplace = True)

In [16]:
rdf.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True


In [17]:
rdf.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype='object')

In [18]:
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch',
       'embarked']]

In [19]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [20]:
onehot_sex = pd.get_dummies(ndf["sex"], dtype = "int64")
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [21]:
ndf = pd.concat([ndf, onehot_sex], axis = 1)

In [22]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [23]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town", dtype = "int64")
onehot_embarked

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
885,0,1,0
886,0,0,1
887,0,0,1
889,1,0,0


In [24]:
ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [25]:
ndf = pd.concat([ndf, onehot_sex], axis = 1)

In [27]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town", dtype = "int64")
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [28]:
ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1,0,1,0,0,1


In [29]:
ndf.drop(["sex", "embarked"], axis = 1, inplace = True)

In [30]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1,0,1,0,0,1


In [31]:
x = ndf.drop("survived", axis = 1) # 독립변수 x
y = ndf["survived"] # 종속 변수 y

In [32]:
x= ndf.drop('survived', axis = 1)
y = ndf['survived']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify = y, random_state=7)

In [34]:
len(x_train), len(x_test)

(499, 215)

In [35]:
knn = KNeighborsClassifier(n_neighbors = 5)

In [36]:
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [37]:
y_pred = knn.predict(x_test)

In [38]:
pd.DataFrame({"ans" : y_test, "pred": y_pred})

,ans,pred
11,1,1
429,1,1
130,0,0
651,1,1
182,0,0
...,...,...
408,0,0
439,0,0
289,1,1
78,1,1


In [39]:
y_test.value_counts()

survived
0    128
1     87
Name: count, dtype: int64

In [41]:
knn_matrix = confusion_matrix(y_test, y_pred)
knn_matrix

array([[109,  19],
       [ 23,  64]], dtype=int64)

In [42]:
knn_report = classification_report(y_test, y_pred)

In [43]:
print(knn_report)

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       128
           1       0.77      0.74      0.75        87

    accuracy                           0.80       215
   macro avg       0.80      0.79      0.80       215
weighted avg       0.80      0.80      0.80       215



In [44]:
x.head()

,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S,female,male,town_C,town_Q,town_S
0,3,22.0,1,0,0,1,0,0,1,0,1,0,0,1
1,1,38.0,1,0,1,0,1,0,0,1,0,1,0,0
2,3,26.0,0,0,1,0,0,0,1,1,0,0,0,1
3,1,35.0,1,0,1,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,0,1,0,0,1,0,1,0,0,1


In [45]:
ss = StandardScaler()

In [46]:
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [47]:
knn = KNeighborsClassifier(n_neighbors=5)

In [48]:
knn.fit(scaled_train, y_train)

KNeighborsClassifier()

In [49]:
y_pred = knn.predict(scaled_test)

In [50]:
pd.DataFrame({"ans" : y_test, "pred": y_pred})

,ans,pred
11,1,1
429,1,1
130,0,0
651,1,1
182,0,0
...,...,...
408,0,0
439,0,0
289,1,1
78,1,1


In [51]:
knn_matrix = confusion_matrix(y_test, y_pred)
knn_matrix

array([[100,  28],
       [ 20,  67]], dtype=int64)

In [52]:
knn_report = classification_report(y_test, y_pred)
print(knn_report)

              precision    recall  f1-score   support

           0       0.83      0.78      0.81       128
           1       0.71      0.77      0.74        87

    accuracy                           0.78       215
   macro avg       0.77      0.78      0.77       215
weighted avg       0.78      0.78      0.78       215

